# Brief recap on Python generators

Reference: [python-mastery repo](https://github.com/dabeaz-course/python-mastery)

In particular, see [pdf](https://github.com/dabeaz-course/python-mastery/blob/main/PythonMastery.pdf) from page 468 onwards

Credit: "Advanced Python Mastery" by [David Beazley](https://www.dabeaz.com)

In [1]:
from collections.abc import Iterable, Generator

import simpy

In [2]:
def finite_generator(n: int) -> Iterable[int]:
    """
    generate the sequence of first n even numbers
    """

    print("start generation...")
    
    i = 0
    c = 0
    while True:
        if c == n:
            print("end of generation.")
            return
        if i % 2 == 0:
            yield i
            c += 1
        i += 1


In [6]:
print(finite_generator)

<function finite_generator at 0x1201351c0>


In [3]:
# calling a generator function creates a generator object.
# it does not start running the function

fin_gen = finite_generator(n=10)

In [5]:
print(fin_gen)

<generator object finite_generator at 0x120112dc0>


In [4]:
l = list(fin_gen)
print(l)

start generation...
end of generation.
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


In [7]:
# Generators are one-time use
# when the generator hit return, 
# iteration stops and the generator is "consumed"

next(fin_gen)

StopIteration: 

In [8]:
fin_gen = finite_generator(n=10)
print(next(fin_gen))

start generation...
0


In [9]:
print(next(fin_gen))

2


In [10]:
def f():
    fin_gen = finite_generator(n=10)
    while True:
        e = next(fin_gen)
        print(e)

f()

start generation...
0
2
4
6
8
10
12
14
16
18
end of generation.


StopIteration: 

In [11]:
def infinite_generator() -> Iterable[int]:
    """
    Generate the infinite sequence of even numbers 
    """

    i = 0
    while True:
        if i % 2 == 0:
            yield i
        i += 1


In [12]:
inf_gen = infinite_generator()

In [13]:
[next(inf_gen) for _ in range(10)]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [21]:
next(inf_gen)

34

# Simpy

main reference: [Simpy documentation](https://simpy.readthedocs.io/en/latest/)

Simpy is a process-based discrete-event simulation framework based on standard Python.

It provides the modeler with components for building models of simulation systems.

It is event-based, and the simulation time is advanced by the occurrence of events.

The main components of Simpy are:
 - **Environment**: the container for the simulation
 - **Processes**: the active components of the simulation
 - **Resources**: the shared entities that processes can use to coordinate access to limited capacity entities
 - **Events**: the occurrences that drive the simulation forward

The simulation is driven by the **occurrence of events**, which are **scheduled** and processed **by the environment**.

The environment schedules and triggers the events, and manages the simulation time.

The resources are used by processes to coordinate access to shared entities with limited capacity.

The simulation ends when there are no more events to process.

# First Simpy process

Our first example will be a car process. 

The car will alternately drive and park for a while. 

When it starts driving (or parking), it will print the current simulation time.

In [22]:
def car(env):

    # Eternal process (infinite generator, i.e. No return)
    while True:
        
        print(f'Start parking at {env.now}')
        parking_duration = 5
        yield env.timeout(parking_duration)
        
        print(f'Start driving at {env.now}')
        trip_duration = 2
        yield env.timeout(trip_duration)


In [30]:
env = simpy.Environment()
car_inst = car(env)
proc = env.process(car_inst)

In [31]:
type(car_inst)

generator

In [32]:
type(proc)

simpy.events.Process

In [33]:
# run the simulation until timestep 15
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


In [34]:
# resume the simulation and run until timestep 15
env.run(until=30)

Start driving at 19
Start parking at 21
Start driving at 26
Start parking at 28


# Process interaction

Processes can **interact** with each other.

Examples:
 - **waiting** for another process to finish
 - **interrupting** another process

In Simpy, processes are technically **events**. Their duration depends on the process implementation.

## Waiting for a process to finish

In [35]:
class Car:
    def __init__(self, env):
        self.env = env
        
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self) -> Generator[simpy.events.Event, None, None]:

        # Eternal process (infinite generator, i.e. No return)
        while True:
        
            print('Start parking and charging at %d' % self.env.now)
            
            # We yield the process that process() returns
            # to wait for it to finish
            
            charge_duration: int = 5
            yield self.env.process(self.charge(charge_duration))
            
            # The charge process has finished, 
            # and we can start driving again.
            
            print('Start driving at %d' % self.env.now)
            
            trip_duration = 2
            yield self.env.process(self.drive(trip_duration))

    def charge(self, duration: float) -> Generator[simpy.events.Event, None, None]:
        yield self.env.timeout(duration)

    def drive(self, duration: float) -> Generator[simpy.events.Event, None, None]:
        yield self.env.timeout(duration)


In [36]:
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


## Interrupting a process

In [37]:
class Car:
    def __init__(self, env: simpy.Environment, charge_duration: float = 5, trip_duration: float = 2) -> None:
        self.env = env
        self.action = env.process(self.run())
        self.charge_duration = charge_duration
        self.trip_duration = trip_duration
    
    def run(self) -> Generator[simpy.events.Event, None, None]:
        
        # Eternal process (infinite generator, i.e. No return)
        while True:
            
            print('Start parking and charging at %d' % self.env.now)
            
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge())
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')
    
            print('Start driving at %d' % self.env.now)
            yield self.env.process(self.drive())
    
    def charge(self) -> Generator[simpy.events.Event, None, None]:
        yield self.env.timeout(self.charge_duration)

    def drive(self) -> Generator[simpy.events.Event, None, None]:
        yield self.env.timeout(self.trip_duration)


def interrupt_charging(env: simpy.Environment, car: Car):
    yield env.timeout(3)
    car.action.interrupt()


In [38]:
env = simpy.Environment()
car = Car(env)
env.process(interrupt_charging(env, car))
env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


# Shared resources

Simpy *resources* are simulation components that can be used by processes to coordinate access to *shared* simulation entites with **limited capacity**.

## Basic Resource Usage

In [42]:
class Logger:
    last_timestamp: float
    
    def __init__(self, env: simpy.Environment) -> None:
        self.env = env
        Logger.last_timestamp = self.env.now
    
    def log(self, message: str):
        if self.env.now > Logger.last_timestamp:
            print()

        print(f'[{self.env.now}] {message}')
        Logger.last_timestamp = self.env.now

class Car:
    def __init__(self, env: simpy.Environment, i: int, bcs: simpy.Resource) -> None:
        self.env = env
        self.name = f'Car {i}'
        self.color = f'\033[9{i+1}m'
        self.bcs = bcs
        self.logger = Logger(self.env)
        
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def __str__(self):
        return f"{self.color}{self.name}\033[0m"

    def run(self) -> Generator[simpy.events.Event, None, None]:
        while True:
            # Generate a request to use the bcs
            # If the resource is already in use, 
            # the process will be put in queue until the resource is available
            
            self.logger.log(f'{self} requesting the battery charging station')
            
            # context manager
            # see Python Mastery pdf, page 217
            with self.bcs.request() as req:
                
                # Waiting for the bcs to become available
                yield req
                
                yield self.env.process(self.charge(duration=5))
                
            yield self.env.process(self.drive(duration=2))
                
    def charge(self, duration: float) -> Generator[simpy.events.Event, None, None]:
        self.logger.log(f'{self} starting to charge')
        yield self.env.timeout(duration)
        self.logger.log(f'{self} leaving the battery charging station')
    
    def drive(self, duration: float) -> Generator[simpy.events.Event, None, None]:
        self.logger.log(f'{self} starting to drive')
        yield self.env.timeout(duration)
        # self.logger.log(f'{self} leaving the battery charging station')


In [43]:
env = simpy.Environment()
battery_charging_station = simpy.Resource(env, capacity=2)

cars = [Car(env, i, battery_charging_station) for i in range(4)]

In [44]:
env.run(until=30)

[0] Car 0 requesting the battery charging station
[0] Car 1 requesting the battery charging station
[0] Car 2 requesting the battery charging station
[0] Car 3 requesting the battery charging station
[0] Car 0 starting to charge
[0] Car 1 starting to charge

[5] Car 0 leaving the battery charging station
[5] Car 1 leaving the battery charging station
[5] Car 0 starting to drive
[5] Car 1 starting to drive
[5] Car 2 starting to charge
[5] Car 3 starting to charge

[7] Car 0 requesting the battery charging station
[7] Car 1 requesting the battery charging station

[10] Car 2 leaving the battery charging station
[10] Car 3 leaving the battery charging station
[10] Car 2 starting to drive
[10] Car 3 starting to drive
[10] Car 0 starting to charge
[10] Car 1 starting to charge

[12] Car 2 requesting the battery charging station
[12] Car 3 requesting the battery charging station

[15] Car 0 leaving the battery charging station
[15] Car 1 leaving the battery charging station
[15] Car 0 starti